###  Data Set

In [1]:
# R consists of 15 pairs, each comprising two attributes (nominal and numeric)
R = [('Adele',8),('Bob',22),('Clement',16),('Dave',23),('Ed',11),
     ('Fung',25),('Goel',3),('Harry',17),('Irene',14),('Joanna',2),
     ('Kelly',6),('Lim',20),('Meng',1),('Noor',5),('Omar',19)]

# S consists of 8 pairs, each comprising two attributes (nominal and numeric)
S = [('Arts',8),('Business',15),('CompSc',2),('Dance',12),('Engineering',7),
     ('Finance',21),('Geology',10),('Health',11),('IT',18)]


### 1. Serial Join Algorithms

Let's first understand serial join algorithms - join algorithms implemented in nonparallel machines. Parallel join algorithms adopt a data partitioning parallelism approach, whereby parallelism is achieved through data partitioning. That is, a join operation implemented on each processor would employ a serial join algorithm. In Section 2, we will learn more about parallel join algorithms.

In this activity, we will consider the following three serial join algorithms:

   - Nested-loop join algorithm,
   - Sort-merge join algorithm,
   - Hash-based join algorithm

##### 1.1 Nested-Loop Join Algorithm

Nested-loop join is the simplest form of join algorithm. For each record of the first table, it goes through all records of the second table. This is repeated for all records of the first table. It is called a nested loop because it consists of two levels of loops: inner loop (looping for the second table) and outer loop (looping for the first table).

Exercise: Undertand and run the nested-loop join algorithm using the join attribute - the numeric attribute in two tables R and S. Then, discuss the time complexity of this algorithm as well as its pros and cons.


In [2]:
def NL_join(T1, T2):
    """
    Perform the nested-loop join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    result = []
    
    for r1 in T1:
        for r2 in T2:
            if r1[1] == r2[1]:
                result.append({", ".join([r1[0], str(r1[1]), r2[0]])})
    return result


In [3]:
NL_join(R,S)

[{'Adele, 8, Arts'}, {'Ed, 11, Health'}, {'Joanna, 2, CompSc'}]


### 1.2 Sort-Merge Join Algorithm

Sort-merge join is based on sorting and merging operations. The first step of joining is to sort the two tables based on the joining attribute in an ascending order, and the second step is merging the two sorted tables. If the value of the joining attribute in R is smaller than that in S, it skips to the next value of the joining attribute in R. On the other hand, if the value of the joining attribute in R is greater than that in S, it skips to the next value of the joining attribute in S. When the two values match, the two corresponding records are concatenated and placed into the query result.

Exercise: Complete the sort-merge join algorithm based on the above definition by implementing the following code block between '### START CODE HERE ###' and '### END CODE HERE ###'. Discuss the time complexity of this algorithm in terms if its efficiency. Also, compare it with the nest-loop join algorithm.


In [4]:
def SM_join(T1, T2):
    """
    Perform the sort-merge join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    result = []
    
    # sort T1 based on the join attribute
    s_T1 = list(T1)
    s_T1 = sorted(s_T1, key=lambda s_T1: s_T1[1])
    
    # sort T2 based on the join attribute
    s_T2 = list(T2)
    s_T2 = sorted(s_T2, key=lambda s_T2: s_T2[1])
   
    ### START CODE HERE ### 
    i = j = 0
    while i < len(s_T1) and j < len(s_T2):#the original sample is inccorect which leads to later the missing of 8 Adele
        r = s_T1[i][1]
        s = s_T2[j][1]
        # If join attribute s_T1(i) < join attribute s_T2(i)
        if r < s:
            i += 1
        
        # else 
        else:
            if r == s:
                result.append({', '.join([s_T1[i][0],str(s_T1[i][1]),s_T2[j][0]])})
                i += 1
                j += 1
            # if join attribute s_T1(1) > join attribute s_T2(1)
            # #---Implement here
            
            # else 
            else:
                # put records s_T1(i) and s_T2(j) into the result and i++, j++
                # #---Implement here
                j += 1

    ### END CODE HERE ###

    return result


In [5]:
SM_join(R,S)

[{'Joanna, 2, CompSc'}, {'Adele, 8, Arts'}, {'Ed, 11, Health'}]


### 1.3 Hash-Based Join Algorithm

A hash-based join is basically made up of two processes: hashing and probing. A hash table is created by hashing all records of the first table using a particular hash function. Records from the second table are also hashed with the same hash function and probed. If any match is found, the two records are concatenated and placed in the query result.

A decision must be made about which table is to be hashed and which table is to be probed. Since a hash table has to be created, it would be better to choose the smaller table for hashing and the larger table for probing.

Exercise: Complete the hash-based join algorithm by implementing the following code block between '### START CODE HERE ###' and '### END CODE HERE ###'. Discuss the time complexity of this algorithm in terms if its efficiency. Also, compare it with the above two join algorithms.


In [6]:
def H(r,n=2):
    """
    We define a hash function 'H' that is used in the hashing process works 
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute. 
    
    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """
    #digits = [int(d) for d in str(r[1])] is not valid according to the decription of the method which assume the index
    #has no more than 2 digits
    digits = []
    digits = [int(d) for d in str(r[1])[:n]]
    return sum(digits)

In [7]:
H(R[0])

8

In [8]:
def HB_join(T1, T2):
    """
    Perform the hash-based join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    result = []
    dic = {}
    
    for s in T1:
        s_key = H(s)
        if s_key not in dic:
            dic[s_key] = {s}
        else:
            dic[s_key].add(s)

    for r in T2:
        r_key = H(r)
        if r_key in dic:
            for s in dic[r_key]:
                if r[1] == s[1]:
                    joint = {", ".join([s[0],str(s[1]),r[0]])}
                    result.append(joint)
    return result


In [9]:
HB_join(R,S)

[{'Adele, 8, Arts'}, {'Joanna, 2, CompSc'}, {'Ed, 11, Health'}]


### 2 Parallel Join Algorithms

Parallelism of join queries is achieved through data parallelism, whereby the same task is applied to different parts of the data. That is, after data partitioning has been completed, each processor will have its own data to work with. Thus, each processor will apply any serial join algorithm. Once this is carried out in each processor, the final results of the join operation are consolidated from the results obtained from different processors.

We now look into the following two parallel join algorithms:

   - Divide and Broadcast-Based Parallel Join Algorithms
   - Disjoint Partitioning-Based Parallel Join Algorithms

#### 2.1 Divide and Broadcast-Based Parallel Join Algorithms

These algorithms consist of two stages: (1) data partitioning using the divide and broadcast method and (2) a local join.

The divide and broadcast data partitioning method consists of dividing one table into multiple disjoint partitions, where each partition is allocated a processor, and broadcasts the other table to all available processors. Dividing one table may be done simply by using equal division, so that each partition will have the same size, whereas broadcasting is actually replicating the content of the second table to all processors. Thus it is preferable for the smaller table to be broadcast and the larger table to be divided.

Exercise: Understand how a divide and broadcast-based parallel join algorithms works given the tables R and S above. We assume that the hash-based join algorithm (i.e. HB_join(.)) are used (see the above) and the round-robin data partitioning function that designed for "Parallel Search" acitivity (i.e. see below: rr_partition(.)) is used to implement this parallel join algorithm. Also, we assume that we use a shared-memory architecture is used, so there is no replication of the broadcast table S.


In [10]:
# Round-robin data partitionining function
def rr_partition(data, n):
    """
    Perform data partitioning on data

    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors

    Return:
    result -- the paritioned subsets of D
    """
    result = []
    #use a list of lists, each inner list is a partiton of the dataset
    for i in range(n):
        result.append([])
    
    ### START CODE HERE ### 
    
    # Calculate the number of the elements to be allocated to each bin
    n_bin = len(data)/n
    
    # For each bin, perform the following
    for index, element in enumerate(data): 
        # Calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)
        #print(str(index) + ":" + str(element))
        result[index_bin].append(element)
    ### END CODE HERE ###
    
    return result

In [11]:
rr_partition(R,3)

[[('Adele', 8), ('Dave', 23), ('Goel', 3), ('Joanna', 2), ('Meng', 1)],
 [('Bob', 22), ('Ed', 11), ('Harry', 17), ('Kelly', 6), ('Noor', 5)],
 [('Clement', 16), ('Fung', 25), ('Irene', 14), ('Lim', 20), ('Omar', 19)]]

In [12]:
import multiprocessing as mp
def DDP_join(T1, T2, n_processor):
    """
    Perform a divide and broadcast-based parallel join algorithms.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined
    n_processor -- the number of parallel processors

    Return:
    result -- the joined table
    """
    
    result = []
    
    T1_subsets = rr_partition(T1,n_processor)
    pool = mp.Pool(processes=n_processor)
    
    for t1 in T1_subsets:
        result.append(pool.apply(HB_join,[t1,T2]))
    return result

In [13]:
DDP_join(R,S,4)

[[{'Adele, 8, Arts'}, {'Ed, 11, Health'}], [{'Joanna, 2, CompSc'}], [], []]


### 2.2 Disjoint Partitioning-Based Parallel Join Algorithms

These algorithms also consist of two stages: a data partitioning stage using a disjoint partitioning and a local join. For the data partitioning, a disjoint partitioning, such as range partitioning or hash partitioning, may be used.

Exercise: Complete the following a disjoint partitioning-based parallel join algorithm.

Use all the three serial join algorithms above, and see whether the joined results are the **```same or not```**:

   - **Nested-loop join algorithm**
   - **Sort-merge join algorithm**
   - **Hash-based join algorithm**

As a data partitioning method, use the range partitioninig method that we provided for "Parallel Search" acitivity (i.e. range_partition(.)). Assume that we have 3 parallel processors, processor 1 will get records with join attribute value between 1 and 9, processor 2 between 10 and 19, and processor 3 between 20 and 29. Note that you need to modify this function in the way that it partitions the table on the join attribute.

**`Note that both tables R and S need to be partitioned based on the join attribute with the same range partitioning function.`**


In [14]:
# Range data partitionining function (Need to modify as instructed above)
def range_partition(data, range_indices):
    """
    Perform range data partitioning on data based on the join attribute

    Arguments:
    data -- an input dataset which is a list
    range_indices -- the index list of ranges to be split

    Return:
    result -- the paritioned subsets of D
    """
    result = []
    #try using generator to just search the who dataset once without needing to slice
    def new_data():
        for x in sorted(data,key= lambda x: x[1]):
            yield x
    dataSet = new_data()
    
    #number of bins -1
    n = len(range_indices) + 1
    s =[]
    for j in range(n):
        s.append([])
    for i in range(n-1):
        for x in dataSet:
            if x[1] >= range_indices[i]:
                s[i+1].append(x)
                break
            s[i].append(x)
        result.append(s[i])
    s_last = s[n-1] + ([x for x in dataSet])
    result.append(s_last)
    return result

In [15]:
range_partition(R, [10, 20])

[[('Meng', 1),
  ('Joanna', 2),
  ('Goel', 3),
  ('Noor', 5),
  ('Kelly', 6),
  ('Adele', 8)],
 [('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19)],
 [('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]]

In [16]:
range_partition(S, [10, 20])

[[('CompSc', 2), ('Engineering', 7), ('Arts', 8)],
 [('Geology', 10),
  ('Health', 11),
  ('Dance', 12),
  ('Business', 15),
  ('IT', 18)],
 [('Finance', 21)]]

In [17]:
def DPBP_join(T1, T2, n_processor,join_type):
    """
    Perform a disjoint partitioning-based parallel join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined
    n_processor -- the number of parallel processors

    Return:
    result -- the joined table
    """
    
    result = []
    
    ### START CODE HERE ### 
    pool = mp.Pool(processes=n_processor)
    #--- Implement the algorithm

    # !!Partition T1 & T2 into sub-tables using range_partition().
    # The number of the sub-tables must be the equal to the n_processor
    T1_subsets = range_partition(T1,[10,20])
    T2_subsets = range_partition(T2,[10,20])
    # Apply local join for each processor
    for i in range(n_processor):
        result.append(pool.apply(join_type,[T1_subsets[i],T2_subsets[i]]))
    ### END CODE HERE ###
    
    return result

In [18]:
#test output of all three local join methods
a = DPBP_join(R,S,3,SM_join)
b = DPBP_join(R,S,3,HB_join)
c = DPBP_join(R,S,3,NL_join)
print(a,"Sort_merge_join")
print(a,"Hash_based_join")
print(a,"Nested_loop_join")

([[set(['Joanna, 2, CompSc']), set(['Adele, 8, Arts'])], [set(['Ed, 11, Health'])], []], 'Sort_merge_join')
([[set(['Joanna, 2, CompSc']), set(['Adele, 8, Arts'])], [set(['Ed, 11, Health'])], []], 'Hash_based_join')
([[set(['Joanna, 2, CompSc']), set(['Adele, 8, Arts'])], [set(['Ed, 11, Health'])], []], 'Nested_loop_join')


### Test of inner function and generator (which can remember its formmer state when temporaryly exit the fution)

In [19]:
def x(anumber):
    def a():
        print(anumber)
    a()

In [20]:
def gen(aList):
    for x in aList:
        yield x

In [21]:
a =gen([1,2,3])

In [22]:
r = []
r.append([x for x in a])

In [23]:
r

[[1, 2, 3]]

In [24]:
def x():
    for a in [1,2,3,4,5]:
        if a%2 == 0:
            yield a
            

In [25]:
n = x()

In [26]:
for a in n:
    print(a)

2
4
